In [ ]:
import folium
import pandas as pd
import geopandas as gpd
import json

# Load the Excel file into a Pandas DataFrame
df = pd.read_excel('data.xlsx')

# Convert the DataFrame to a GeoDataFrame and set the CRS (Coordinate Reference System)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LON, df.LAT), crs="EPSG:4326")
estimate_crs = gdf.estimate_utm_crs()
gdf = gdf.to_crs(estimate_crs)

# Add a buffer in meters to the geometries
gdf['geometry'] = gdf['geometry'].buffer(2000, 16)
gdf = gdf.to_crs({'init': 'epsg:4326'})

# Save the GeoDataFrame to a GeoJSON file
gdf.to_file("output.geojson", driver="GeoJSON")

# Create a Folium map
m = folium.Map(location = [55.750066, 37.614356], zoom_start=10)

# Add the GeoJSON data to the map
folium.GeoJson(
    gdf,
    style_function=lambda feature: {
        "fillColor": "#ff0000",
        "fill-opacity": 0.3,
        "stroke": False
}
    ).add_to(m)

# Save the map as an HTML file
m.save('geo.html')